# Grid Search

In [12]:
import numpy as np
import pandas as pd
import lightgbm as lgb

In [13]:
dataset = pd.read_csv('TRAIN_BANKRUPTCY.csv')
#X = dataset.iloc[:, :-1].values
#y = dataset.iloc[:, -1].values

In [14]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
X1=dataset.iloc[:,:-1]
y1=dataset.iloc[:,-1]
X_os,y_os=sm.fit_resample(X1,y1)

In [15]:
X=X_os.values
y=y_os.values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [28]:
    #from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [ ]:
#from sklearn.svm import SVC
#classifier = SVC(kernel = 'rbf', random_state = 0)
#absclassifier.fit(X_train, y_train)

In [17]:
from lightgbm import LGBMClassifier
classifier = LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
classifier.fit(X_train,y_train)

C:\Users\mpadmara\Anaconda3\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(metric='None', n_estimators=5000, n_jobs=4, random_state=314,
               silent=True)

In [6]:
#from xgboost import XGBClassifier
#classifier = XGBClassifier(booster='dart',n_estimators=600,max_depth=10)
#classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=600,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[3689   16]
 [   1 3645]]


0.9976873894708203

In [19]:
xg_probs = classifier.predict_proba(X_test)
xg_probs_final=xg_probs[:,1]
xg_probs_final.shape

(7351,)

In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, xg_probs_final)

0.9999967427746971

In [22]:
test = pd.read_csv('bankruptcy_Test_X.csv')
X_TEST=test.iloc[:,1:].values
#X_testfinal=sc.transform(X_TEST)
X_TEST.shape

(5000, 64)

In [23]:
prob=classifier.predict_proba(X_TEST)
prob.shape
df = pd.DataFrame(prob, columns =['ID','class'])
df.to_excel("score21.xlsx")

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
parameters = [{'max_depth': [2, 5, 10, 1], 'n_estimators': [60,220,150],'scale_pos_weight':[40,50,60]},
              {'learning_rate': [0.3, 0.03, 0.75, 0.05], 'colsample_bytree': [ 0.3, 0.5 , 0.8 ], 'reg_alpha': [0, 0.5, 1, 5], 'reg_lambda': [0, 0.5, 1, 5]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 98.19 %
Best Parameters: {'max_depth': 10, 'n_estimators': 60, 'scale_pos_weight': 40}
